# Using entities to structure code and integrate existing VHDL designs

## instantiating entities

CoHDL designs can be made up of more than one entity.

* each entity is compiled once into a VHDL entity which is then instantiated
* entities are limited to ports, functions and classes are more flexible and can take arbitrary Python objects as arguments
* external entities provide an interface to instantiate existing VHDL designs

Since Python classes and functions are also available it is usually not necessary to use entities to structure code.

In [ ]:
from __future__ import annotations

from cohdl import Entity, Port, Bit
from cohdl import std

class Inverter(Entity):
    input = Port.input(Bit)
    output = Port.output(Bit)

    def architecture(self):
        @std.concurrent
        def logic():
            self.output <<= ~self.input

# the top level entity of this design
# used made up of two instances of Inverter
class UseInverters(Entity):
    inp_a = Port.input(Bit)
    out_a = Port.output(Bit)
    
    inp_b = Port.input(Bit)
    out_b = Port.output(Bit)

    def architecture(self):
        # entities are instantiated by passing all ports
        # as keyword parameters to the class constructor
        Inverter(input=self.inp_a, output=self.out_a)
        
        Inverter(input=self.inp_b, output=self.out_b)

print(std.VhdlCompiler.to_string(UseInverters))

## external entities

Entities can be marked as external to import VHDL entities into CoHDL designs.


In [ ]:
from __future__ import annotations

from cohdl import Entity, Port, Bit
from cohdl import std

# declaration of some entity that is defined
# in vhdl
class MyVhdlInverter(Entity, extern=True):
    input = Port.input(Bit)
    output = Port.output(Bit)

    # external entities have no architecture method
    # they only define ports so CoHDL can generate the
    # corresponding entity instantiations

class UseInverters(Entity):
    inp_a = Port.input(Bit)
    out_a = Port.output(Bit)
    
    inp_b = Port.input(Bit)
    out_b = Port.output(Bit)

    def architecture(self):
        # instantiating external entities works the same
        # way as normal CoHDL entities
        MyVhdlInverter(input=self.inp_a, output=self.out_a)
        MyVhdlInverter(input=self.inp_b, output=self.out_b)

print(std.VhdlCompiler.to_string(UseInverters))